In [ ]:
import numpy as np
from astropy.io import fits
import moment_utils as mutil

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, SymLogNorm, Normalize
plt.rcParams['figure.figsize'] = [12, 10]
SMALL_SIZE = 14
MEDIUM_SIZE = 18
BIGGER_SIZE = 22
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


def plot_spectrum(spectrum, camera_wavelengths, title=None, y_label=None, outfile=None):
    frequencies = 3.0e10 / camera_wavelengths
    # convert to GHz
    frequencies /= 1.0e9

    plt.plot(frequencies, spectrum)
    plt.title(title)
    plt.xlabel("Frequency (GHz)")
    if y_label is None:
        y_label = "Specific Intensity (Kelvin)"
    plt.ylabel(y_label)
    plt.show()
    
    if outfile:
        plt.savefig(outfile)
        print("Spectrum saved at {}".format(outfile))
        

def plot_moment_map(moment_map, moment, normed=False, scale_dimensions=None,
                    scale_units=None, outfile=None, colorscale=LogNorm(),
                    title=None, cbar_label=None):
    if scale_dimensions is None:
        scale_dimensions = moment_map.shape
    if scale_units is None:
        scale_units = "px"
    x_scale, y_scale = scale_dimensions
    x_lo, x_hi, y_lo, y_hi = -x_scale/2, x_scale/2, -y_scale/2, y_scale/2
    npix_y, npix_x = moment_map.shape
    x_range = np.linspace(x_lo, x_hi, npix_x)
    y_range = np.linspace(y_lo, y_hi, npix_y)
    
    plt.pcolormesh(x_range, y_range, moment_map, cmap='viridis', norm=colorscale)
    cbar = plt.colorbar()
    plt.axis((x_lo, x_hi, y_lo, y_hi))
    plt.xlabel("X ({})".format(scale_units))
    plt.ylabel("Y ({})".format(scale_units))
    if normed:
        cbar.set_label(r'[(km/s)^{}]'.format(moment), rotation=90)
        plt.title(r'Normalized Integrated Temperature $\int T_v v^{} dv$'.format(moment))
    else:
        cbar.set_label(r'[K (km/s)^{}]'.format(1+moment), rotation=90)
        plt.title(r'Integrated Temperature $\int T_v v^{} dv$'.format(moment))
    if title is not None:
        plt.title(title)
    if cbar_label is not None:
        cbar.set_label(cbar_label, rotation=90)

    if outfile:
        plt.savefig(outfile)
        print("Moment map saved at {}".format(outfile))
        
    plt.show()

In [ ]:
full_mom2 = mutil.read_moment_map("co_emission/full-mom2.fits")
tracer_mom2 = mutil.read_moment_map("co_emission/tracer-mom2.fits")
nontracer_mom2 = mutil.read_moment_map("co_emission/nontracer-mom2.fits")
obs_outflow_mom2 = mutil.read_moment_map("co_emission/obs-outflow-mom2.fits")
obs_nonoutflow_mom2 = mutil.read_moment_map("co_emission/obs-nonoutflow-mom2.fits")

nontracer_2ratio = nontracer_mom2/full_mom2
tracer_2ratio = tracer_mom2/full_mom2
obs_outflow_2ratio = obs_outflow_mom2/full_mom2
obs_nonoutflow_2ratio = obs_nonoutflow_mom2/full_mom2

In [ ]:
maps = [full_mom2, tracer_mom2, nontracer_mom2, obs_outflow_mom2, obs_nonoutflow_mom2]
titles = ["Full 2-mom", "Tracer 2-mom", "Nontracer 2-mom", "Observer Outflow 2-mom",
          "Observer Nonoutflow 2-mom"]
outfiles = ["full-mom2", "tracer-mom2", "nontracer-mom2", "obs-outflow-mom2", "obs-nonoutflow-mom2"]
outfiles = ["co_emission/results/{}".format(fn) for fn in outfiles]

CM2PC = 3.2407792896664e-19
scale = 8e17*CM2PC, 8e17*CM2PC
for i in range(len(maps)):    
    plot_moment_map(maps[i], moment=2, scale_dimensions=scale, scale_units="pc",
                    colorscale=SymLogNorm(linthresh=1e-2), title=titles[i],
                    outfile=outfiles[i])

In [ ]:
maps = [tracer_2ratio, nontracer_2ratio, obs_outflow_2ratio, obs_nonoutflow_2ratio]
titles = ["Tracer/Full 2-mom ratio", "Nontracer/Full 2-mom ratio",
          "ObserverOutflow/Full 2-mom ratio", "ObserverNonoutflow/Full 2-mom ratio"]
outfiles = ["ratio-tracer-mom2", "ratio-nontracer-mom2", "ratio-obs-outflow-mom2",
            "ratio-obs-nonoutflow-mom2"]
outfiles = ["co_emission/results/{}".format(fn) for fn in outfiles]

CM2PC = 3.2407792896664e-19
scale = 8e17*CM2PC, 8e17*CM2PC
for i in range(len(maps)):    
    plot_moment_map(maps[i], moment=2, scale_dimensions=scale, scale_units="pc",
                    colorscale=Normalize(), title=titles[i],
                    cbar_label="[unitless]", outfile=outfiles[i])

In [ ]:
spectrum = mutil.create_spectrum(temp_cube, moment_map=moment_map)
plot_spectrum(spectrum, camera_wavelengths, title="Spectrum @ brightest pixel")